In [1]:
import re
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import time
import sqlite3
from urllib.parse import quote_plus
import yaml
from datetime import datetime
import os
from KBDownloader import search_swedish_newspapers, fetch_newspaper_data, save_checkpoint, load_checkpoint


# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Assign variables from the YAML configuration
venue_list = config['venue_list']
start_year = config['start_year']
years_to_crawl = config['years_to_crawl']
newspaper = config['newspaper']
db_path = config['db_path']

# Ensure the database file exists
if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    conn.close()

# Load checkpoint if it exists
checkpoint = load_checkpoint()
start_year = checkpoint['year'] if checkpoint else start_year
start_half = checkpoint['half'] if checkpoint else 0
start_index = checkpoint['index'] if checkpoint else 0

# Print out all the settings from the YAML configuration file
print("Configuration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")

#Load the venue list
df = pd.read_excel(venue_list)


Configuration Settings:
venue_list: Datasets/test_lokalerna.xlsx
start_year: 1908
years_to_crawl: 1
newspaper: Dagens nyheter
prompt_filepath: oldtimey_touringbot_prompt_for_deployment.txt
db_path: Datasets/30.06.test.db
llm_model: gpt-3.5-turbo
max_tokens: 1000


In [2]:
# Main loop
for year in range(start_year, start_year + years_to_crawl):
    for half in range(2):
        if year == start_year and half < start_half:
            continue
        
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)

        for index, row in df.iloc[start_index:].iterrows():
            query = row['Lokal']
            safe_query = "".join([c if c.isalnum() else "_" for c in query])

            
            try:
                result = fetch_newspaper_data(
                    query=query,
                    from_date=from_date.strftime('%Y-%m-%d'),
                    to_date=to_date.strftime('%Y-%m-%d'),
                    newspaper=newspaper,
                    config=config,
                    db_path=db_path
                )
                
                if result.get('success'):
                    print(f"Processed query '{query}' successfully.")
                else:
                    print(f"Failed to process query '{query}': {result.get('message')}")
                
                # Save checkpoint after each query, successful or not
                save_checkpoint(year, half, index + 1)
                
            except Exception as e:
                print(f"Error processing query '{query}': {str(e)}")
                save_checkpoint(year, half, index)
                raise  # Re-raise the exception to stop the script

        print(f"Waiting so KB does not get mad. Currently at {from_date} to {to_date}")
        time.sleep(60)
        start_index = 0  # Reset start_index for the next half-year

    start_half = 0  # Reset start_half for the next year

print("All queries processed for all specified years.")

Processed query 'Kungliga teater' successfully.
Checkpoint saved: Year 1908, Half 0, Index 5
Failed to process query 'F.d. Kirsteinska huset (vid Clara)': No data to save. This may be because there are no results for this period.
Checkpoint saved: Year 1908, Half 0, Index 6
Processed query 'Hotel W6 resp. Hotel Continental' successfully.
Checkpoint saved: Year 1908, Half 0, Index 7
Failed to process query 'La Croix mindre salong': No data to save. This may be because there are no results for this period.
Checkpoint saved: Year 1908, Half 0, Index 8
Failed to process query 'Södra teatern i Stadshuset': No data to save. This may be because there are no results for this period.
Checkpoint saved: Year 1908, Half 0, Index 9
Processed query 'Stadshus salongen' successfully.
Checkpoint saved: Year 1908, Half 0, Index 10
Failed to process query 'Stora börssalen': No data to save. This may be because there are no results for this period.
Checkpoint saved: Year 1908, Half 0, Index 11
Processed q

KeyboardInterrupt: 

# View SQL Database

In [3]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# SQL query to select all columns except 'Raw API Result'
query = """
SELECT Date, [Package ID], Part, Page, [ComposedBlock Content], [Full Prompt]
FROM newspaper_data
"""

# Read the query results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
print(df.head())

# Display basic information about the DataFrame
print(df.info())

# Optional: If you want to see all rows, you can use:
# pd.set_option('display.max_rows', None)
# print(df)

# Optional: If you want to save this to a CSV file for further analysis:
# df.to_csv('newspaper_data_summary.csv', index=False)

         Date    Package ID  Part  Page  \
0  1908.05.10  dark-3693840     1     2   
1  1908.02.09  dark-3693618     1     1   
2  1908.02.09  dark-3693618     1     1   
3  1908.02.09  dark-3693618     1     1   
4  1908.02.09  dark-3693618     1     1   

                               ComposedBlock Content  \
0  Svenska sångare och sångerskor i Berlin.\nOpei...   
1  Praktiserande Läkare:\nfBBEnsHBBBMK:.\n'jManMo...   
2  Stockholms-Nyheter*\n/&gt;p ntinisteriella äre...   
3  €arl Fredrik hundqmst.\nV rits Arlborg.\nNdr A...   
4  Kakelugns-Kamin\n"West a"\nkr. 13: 50\nfärdig-...   

                                         Full Prompt  
0  {"custom_id": "dark-3693840-1-2-1", "method": ...  
1  {"custom_id": "dark-3693618-1-1-1", "method": ...  
2  {"custom_id": "dark-3693618-1-1-1", "method": ...  
3  {"custom_id": "dark-3693618-1-1-1", "method": ...  
4  {"custom_id": "dark-3693618-1-1-1", "method": ...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
